In [ ]:
!git clone https://github.com/rmdluo/TartanHacks2023.git

In [ ]:
!unzip TartanHacks2023/data.zip

In [ ]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras

import cv2

In [ ]:
#data loading
shape = (224, 224)
train_ds = tf.keras.utils.image_dataset_from_directory("garbage_classification", image_size=shape, validation_split=0.2,
  subset="training", seed=123)
val_ds = tf.keras.utils.image_dataset_from_directory("garbage_classification", image_size=shape, validation_split=0.2,
  subset="validation", seed=123)

classes = train_ds.class_names

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from keras import Sequential, layers

data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=shape+(3,)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=shape+(3,),
    include_top=False)

base_model.trainable = False

inputs = keras.Input(shape=shape+(3,))
x = data_augmentation(inputs)
x = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)(x)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(12)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# train model
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(patience = 2, verbose = 1, monitor='val_accuracy' , mode='max', min_delta=0.001, restore_best_weights = True)
callbacks = [early_stop]

hist = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=callbacks).history

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

plt.show()

In [ ]:
base_model.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

epochs = 10
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds).history

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

plt.show()

In [ ]:
# test the model
img = cv2.imread("garbage_classification/battery/battery44.jpg")
img = cv2.resize(img, shape)
prediction = model.predict(np.expand_dims(img, axis=0))
print(classes[np.argmax(prediction)])

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# load back in model
interpreter = tf.lite.Interpreter(model_path="model_75.tflite")
classify_lite = interpreter.get_signature_runner('serving_default')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

In [ ]:
# test the lite model
img = cv2.imread("garbage_classification/biological/biological124.jpg")
img = cv2.resize(img, shape)

predictions_lite = classify_lite(input_2=np.float32(np.expand_dims(img, 0)))['dense']
score_lite = tf.nn.softmax(predictions_lite)
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(classes[np.argmax(score_lite)], 100 * np.max(score_lite))
)

predictions = model.predict(np.float32(np.expand_dims(img, 0)))
score = tf.nn.softmax(predictions)
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# save the classes
label_f = open("classes.txt", mode="w")

for c in classes:
  label_f.write(c+"\n")

label_f.close()

In [ ]:
from google.colab import files
files.download("model.tflite")
files.download("classes.txt")